In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
from scipy.stats import uniform, loguniform
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Load dataset
data_path = r'E:\OneDrive\School\Technical Medicine\TM Jaar 1\Machine learning\TM10007_Machine_Learning_Group_4\TM\ecg_data.csv'
data = pd.read_csv(data_path, index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns) - 1}')  # Excluding label column

# Extract features and labels
X = data.iloc[:, :-1].values  # All columns except the last one
y = data.iloc[:, -1].values   # Last column as labels

# Scale the features
scaler = RobustScaler(quantile_range=(25, 75))
X_scaled = scaler.fit_transform(X)

# Define outer and inner loop random seeds
outer = range(0, 5)  # Outer loop (train-test variations)
inner = range(0, 5)  # Inner loop (hyperparameter tuning)

best_params_list = {}  # Stores best params per (outer, inner) iteration
hyperparam_accuracies = {}  # Stores accuracies per hyperparameter set

# Generate a fixed set of random hyperparameter sets (same for all outer loops)
fixed_hyperparam_sets = []
for _ in range(10):  # Generate 10 random hyperparameter sets
    param_set = {
        'C': loguniform(0.01, 1000).rvs(),
        'gamma': loguniform(0.0001, 100).rvs(),
        'class_weight': random.choice(['balanced', None]),
        'shrinking': random.choice([True, False]),
        'tol': uniform(1e-5, 1e-2).rvs()
    }
    fixed_hyperparam_sets.append(param_set)
    hyperparam_accuracies[str(param_set)] = []  # Initialize list for accuracy tracking

# Outer loop for train-test split variations
for outer_rand in tqdm(outer, desc='Outer Loop'):
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=outer_rand, stratify=y  # Maintain class proportions
    )

    # Apply PCA to reduce dimensionality (fit only on training data)
    pca = PCA(n_components=0.99)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Apply SMOTE to balance the training set
    smote = SMOTE(random_state=42)  # Initialize SMOTE
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)  # Apply SMOTE only to training data

    print(f"Explained Variance by PCA Components: {pca.explained_variance_ratio_.sum()}")
    print(f"Resampled training set size: {len(y_train_resampled)}")

    # Inner loop for hyperparameter tuning
    for inner_rand, param_set in tqdm(enumerate(fixed_hyperparam_sets), desc='Inner Loop', leave=True):
        X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
            X_train_resampled, y_train_resampled, test_size=0.15, random_state=inner_rand, stratify=y_train_resampled  # Maintain class proportions
        )

        # Initialize SVM model with RBF kernel
        svm = SVC(kernel='rbf', **param_set, probability=True)  # Enable probability for AUC calculation

        # Train and evaluate model
        start_time = time.time()
        svm.fit(X_train_train, y_train_train)
        elapsed_time = time.time() - start_time  # Calculate elapsed time

        # Best model evaluation
        y_pred = svm.predict(X_test_pca)  # Evaluate on the untouched test set
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Compute confusion matrix for sensitivity and specificity
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()  # True negatives, False positives, False negatives, True positives
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)

        # Compute AUC
        auc = roc_auc_score(y_test, svm.predict_proba(X_test_pca)[:, 1])

        # Store results
        param_key = str(param_set)  # Convert to string for dict key
        hyperparam_accuracies[param_key].append(accuracy)  # Store accuracy

        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"] = param_set.copy()
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['accuracy'] = accuracy
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['sensitivity'] = sensitivity
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['specificity'] = specificity
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['auc'] = auc
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['f1-score'] = f1

        print(f"\nOuter {outer_rand}, Inner {inner_rand} -> Hyperparameters: {param_set}")
        print(f"Test Set Accuracy: {accuracy:.4f}")
        print(f"Sensitivity: {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print(f"AUC: {auc:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Time taken: {elapsed_time:.2f} seconds")

        

# Select the best classifier based on the F1 score
best_f1_score = -1
best_classifier = None
best_metrics = {}

for key, val in best_params_list.items():
    f1 = val['f1-score']
    
    # Update if current F1 score is higher
    if f1 > best_f1_score:
        best_f1_score = f1
        best_classifier = key
        best_metrics = val  # Store the metrics of the best classifier

# Print the best classifier based on F1 score
print(f"\n🏆 Best Classifier Based on F1 Score: {best_classifier}")
print(f"F1 Score: {best_f1_score:.4f}")

# Print the detailed metrics of the best classifier
print(f"\nDetailed Metrics of the Best Classifier:")
print(f"Accuracy: {best_metrics['accuracy']:.4f}")
print(f"Sensitivity: {best_metrics['sensitivity']:.4f}")
print(f"Specificity: {best_metrics['specificity']:.4f}")
print(f"AUC: {best_metrics['auc']:.4f}")
print(f"F1 Score: {best_metrics['f1-score']:.4f}")


The number of samples: 827
The number of features: 9000


Outer Loop:   0%|          | 0/5 [00:00<?, ?it/s]

Explained Variance by PCA Components: 0.990035920418206
Resampled training set size: 1088



Outer 0, Inner 0 -> Hyperparameters: {'C': 130.33286049397753, 'gamma': 2.1512378105086873, 'class_weight': 'balanced', 'shrinking': True, 'tol': 0.0045153744222632694}
Test Set Accuracy: 0.8253
Sensitivity: 0.0000
Specificity: 1.0000
AUC: 0.5000
F1 Score: 0.0000
Time taken: 0.36 seconds



Outer 0, Inner 1 -> Hyperparameters: {'C': 0.10436307798221316, 'gamma': 0.24181651889150765, 'class_weight': None, 'shrinking': False, 'tol': 0.007729466138921315}
Test Set Accuracy: 0.8253
Sensitivity: 0.0000
Specificity: 1.0000
AUC: 0.5000
F1 Score: 0.0000
Time taken: 0.31 seconds


...


Outer 4, Inner 6 -> Hyperparameters: {'C': 0.14658386758848999, 'gamma': 0.5680743380225827, 'class_weight': None, 'shrinking': True, 'tol': 0.005750473780560726}
Test Set Accuracy: 0.8253
Sensitivity: 0.0000
Specificity: 1.0000
AUC: 0.5000
F1 Score: 0.0000
Time taken: 0.30 seconds



Outer 4, Inner 7 -> Hyperparameters: {'C': 0.07027322427683022, 'gamma': 0.00026403018549550817, 'class_weight': 'balanced', 'shrinking': False, 'tol': 0.0075674562038352084}
Test Set Accuracy: 0.6265
Sensitivity: 0.4483
Specificity: 0.6642
AUC: 0.6396
F1 Score: 0.2955
Time taken: 0.26 seconds



Outer 4, Inner 8 -> Hyperparameters: {'C': 169.51153730016173, 'gamma': 0.0002617036271360483, 'class_weight': 'balanced', 'shrinking': True, 'tol': 0.00015518031650574527}
Test Set Accuracy: 0.7771
Sensitivity: 0.2414
Specificity: 0.8905
AUC: 0.6796
F1 Score: 0.2745
Time taken: 0.28 seconds


Inner Loop: 10it [00:03,  3.02it/s]
Outer Loop: 100%|██████████| 5/5 [00:27<00:00,  5.54s/it]


Outer 4, Inner 9 -> Hyperparameters: {'C': 20.54149569775089, 'gamma': 0.9477374054189408, 'class_weight': None, 'shrinking': False, 'tol': 0.001020271544622967}
Test Set Accuracy: 0.8253
Sensitivity: 0.0000
Specificity: 1.0000
AUC: 0.5000
F1 Score: 0.0000
Time taken: 0.33 seconds

🏆 Best Classifier Based on F1 Score: Outer 1 - Inner 8
F1 Score: 0.3396

Detailed Metrics of the Best Classifier:
Accuracy: 0.7892
Sensitivity: 0.3103
Specificity: 0.8905
AUC: 0.7025
F1 Score: 0.3396
